# Hyperparameter Sweep on Google Colab

This notebook runs your RL hyperparameter experiments on Colab and saves results to Google Drive.

## Setup Steps:
1. Mount Google Drive
2. Upload your Python files (or load from Drive)
3. Configure hyperparameters
4. Run sweep
5. Generate plots

**Runtime:** Use CPU (RL training doesn't benefit from GPU)

## 1️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set working directory in your Drive
import os
DRIVE_WORKSPACE = '/content/drive/MyDrive/RL_Experiments'
os.makedirs(DRIVE_WORKSPACE, exist_ok=True)
os.chdir(DRIVE_WORKSPACE)

print(f"✓ Working directory: {os.getcwd()}")

## 2️⃣ Upload Your Code Files

**Option A: Upload files directly to Colab**

In [ ]:
from google.colab import files

print("Upload these files from your local machine:")
print("  - connect_four_env.py")
print("  - perspective_state.py")
print("  - rl_agent.py")
print("  - q_learning.py")
print("  - monte_carlo.py")
print("  - sarsa.py")
print("  - train_single_config.py")
print("  - run_hyperparam_sweep.py")
print("  - analyze_hyperparam_results.py")
print("\nClick 'Choose Files' button below...\n")

uploaded = files.upload()

print(f"\n✓ Uploaded {len(uploaded)} files")

**Option B: Load files from a Google Drive folder**

In [ ]:
# If you've already uploaded files to Drive, specify the folder:
CODE_FOLDER = '/content/drive/MyDrive/RL_Code'  # Adjust path

import shutil

# Copy all .py files to working directory
for filename in os.listdir(CODE_FOLDER):
    if filename.endswith('.py'):
        src = os.path.join(CODE_FOLDER, filename)
        dst = os.path.join(DRIVE_WORKSPACE, filename)
        shutil.copy(src, dst)
        print(f"✓ Copied {filename}")

print("\n✓ All files loaded from Drive")

## 3️⃣ Verify Files

In [ ]:
required_files = [
    'connect_four_env.py',
    'perspective_state.py',
    'rl_agent.py',
    'q_learning.py',
    'monte_carlo.py',
    'sarsa.py',
    'train_single_config.py',
    'run_hyperparam_sweep.py',
]

missing = [f for f in required_files if not os.path.exists(f)]

if missing:
    print("❌ Missing files:")
    for f in missing:
        print(f"  - {f}")
else:
    print("✓ All required files present!")

## 4️⃣ Install Dependencies

In [ ]:
!pip install -q pandas numpy matplotlib seaborn tqdm

## 5️⃣ Configure Hyperparameter Sweep

Edit this cell to set what you want to test:

In [ ]:
# Read current config from run_hyperparam_sweep.py
with open('run_hyperparam_sweep.py', 'r') as f:
    content = f.read()

print("HYPERPARAMETER SWEEP CONFIGURATION")
print("="*70)

# Extract key configuration sections
import re

# Find BOARD_CONFIGS
board_match = re.search(r"BOARD_CONFIGS = \[(.*?)\]", content, re.DOTALL)
if board_match:
    print("\nBoard Configurations:")
    # Extract individual board configs
    boards = re.findall(r"\{(.*?)\}", board_match.group(1), re.DOTALL)
    for i, board in enumerate(boards, 1):
        rows = re.search(r"'rows': (\d+)", board)
        cols = re.search(r"'cols': (\d+)", board)
        episodes = re.search(r"'episodes': (\d+)", board)
        label = re.search(r"'label': '([^']+)'", board)
        if rows and cols and episodes and label:
            print(f"  {i}. {label.group(1)}: {rows.group(1)}×{cols.group(1)}, {episodes.group(1)} episodes")

# Find ALGORITHMS
algorithms_match = re.search(r'ALGORITHMS = \[(.*?)\]', content, re.DOTALL)
if algorithms_match:
    print("\nAlgorithms to Test:")
    print(f"  ALGORITHMS = [{algorithms_match.group(1)}]")

# Find ALPHA_VALUES
alpha_match = re.search(r'ALPHA_VALUES = \[(.*?)\]', content, re.DOTALL)
if alpha_match:
    print("\nLearning Rates (α):")
    print(f"  ALPHA_VALUES = [{alpha_match.group(1)}]")

# Find GAMMA_VALUES
gamma_match = re.search(r'GAMMA_VALUES = \[(.*?)\]', content, re.DOTALL)
if gamma_match:
    print("\nDiscount Factors (γ):")
    print(f"  GAMMA_VALUES = [{gamma_match.group(1)}]")

# Find INITIAL_Q_VALUES
initial_q_match = re.search(r'INITIAL_Q_VALUES = \[(.*?)\]', content, re.DOTALL)
if initial_q_match:
    print("\nInitial Q-Values:")
    print(f"  INITIAL_Q_VALUES = [{initial_q_match.group(1)}]")

# Find EPSILON_SCHEDULES
epsilon_match = re.search(r'EPSILON_SCHEDULES = \[(.*?)\]', content, re.DOTALL)
if epsilon_match:
    print("\nEpsilon Decay Schedules:")
    # Parse the schedules
    schedules = re.findall(r'\((.*?)\)', epsilon_match.group(1))
    for i, sched in enumerate(schedules, 1):
        parts = [p.strip().strip("'\"") for p in sched.split(',')]
        if len(parts) >= 4:
            print(f"  {i}. {parts[3]:15s} - start={parts[0]}, end={parts[1]}, decay={parts[2]}")

print("\n" + "="*70)
print("\n💡 To change configuration:")
print("   Option A: Edit run_hyperparam_sweep.py locally and re-upload")
print("   Option B: Edit directly in Colab: !nano run_hyperparam_sweep.py")
print("             (Ctrl+O to save, Ctrl+X to exit)")
print("\n⚠️  Remember: Total runs = boards × algorithms × alphas × gammas × initial_q × epsilon")
print("              Minus any configs filtered by should_skip_config()")

## 6️⃣ Preview Sweep Plan

In [ ]:
# Load and execute config generation to see what will run
import importlib.util

spec = importlib.util.spec_from_file_location("sweep_module", "run_hyperparam_sweep.py")
sweep_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(sweep_module)

configs = sweep_module.generate_configs()

print(f"Total configurations to test: {len(configs)}")
print(f"Estimated time: ~{len(configs) * 2} minutes ({len(configs) * 2 / 60:.1f} hours)")
print(f"\nFirst 5 configurations:")
for i, config in enumerate(configs[:5], 1):
    print(f"  {i}. {config['algorithm']}, α={config['alpha']}, γ={config['gamma']}, ε_label={config['epsilon_label']}")
    
if len(configs) > 5:
    print(f"  ... and {len(configs)-5} more")

## 7️⃣ Run Hyperparameter Sweep

**⚠️ This will take 1-6 hours depending on configuration**

In [ ]:
# Run the sweep
!python run_hyperparam_sweep.py

# Note: The script will ask for confirmation
# Type 'yes' when prompted

## 8️⃣ Check Results Summary

In [ ]:
import pandas as pd

# Load summary
summary_df = pd.read_csv('hyperparam_sweep/summary.csv')

print(f"Total runs completed: {len(summary_df)}")
print(f"\nTop 5 configurations by second-player performance:")
print("="*80)

top5 = summary_df.nlargest(5, 'final_win_rate_second')
for idx, row in top5.iterrows():
    print(f"{row['algorithm']:12s} | α={row['alpha']:4.2f} | γ={row['gamma']:4.2f} | Second: {row['final_win_rate_second']:.1%}")

print("\n" + "="*80)
print("\nFull results saved to: hyperparam_sweep/summary.csv")

## 9️⃣ Generate Analysis Plots

In [ ]:
# Run analysis script
!python analyze_hyperparam_results.py hyperparam_sweep/

print("\n✓ Plots saved to: hyperparam_sweep/analysis/")

## 🔟 Display Plots in Notebook

In [ ]:
from IPython.display import Image, display
import os

plot_dir = 'hyperparam_sweep/analysis'
plots = [
    'algorithm_comparison.png',
    'alpha_effect.png',
    'epsilon_effect.png',
    'convergence_analysis.png'
]

for plot in plots:
    plot_path = os.path.join(plot_dir, plot)
    if os.path.exists(plot_path):
        print(f"\n{'='*60}")
        print(plot)
        print('='*60)
        display(Image(filename=plot_path))
    else:
        print(f"⚠️ Plot not found: {plot}")

## 1️⃣1️⃣ Download Results (Optional)

If you want to download results to your local machine:

In [ ]:
# Zip everything for easy download
!zip -r hyperparam_results.zip hyperparam_sweep/

from google.colab import files
files.download('hyperparam_results.zip')

print("✓ Results packaged for download")

## 1️⃣2️⃣ Verify Everything Saved to Drive

In [ ]:
import os

def get_dir_size(path):
    """Calculate directory size in MB."""
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            if os.path.exists(filepath):
                total += os.path.getsize(filepath)
    return total / (1024 * 1024)  # Convert to MB

sweep_dir = 'hyperparam_sweep'
if os.path.exists(sweep_dir):
    size_mb = get_dir_size(sweep_dir)
    num_runs = len([d for d in os.listdir(sweep_dir) if os.path.isdir(os.path.join(sweep_dir, d)) and d.startswith('run_')])
    
    print(f"✓ Results saved in Google Drive")
    print(f"\nLocation: {os.path.abspath(sweep_dir)}")
    print(f"Total size: {size_mb:.1f} MB")
    print(f"Number of runs: {num_runs}")
    print(f"Average per run: {size_mb/num_runs:.1f} MB")
    
    # Check for summary and plots
    if os.path.exists(os.path.join(sweep_dir, 'summary.csv')):
        print(f"\n✓ summary.csv present")
    
    analysis_dir = os.path.join(sweep_dir, 'analysis')
    if os.path.exists(analysis_dir):
        plots = [f for f in os.listdir(analysis_dir) if f.endswith('.png')]
        print(f"✓ {len(plots)} analysis plots generated")
else:
    print("❌ No results directory found")

## ✅ Complete!

Your results are saved in Google Drive at:
```
MyDrive/RL_Experiments/hyperparam_sweep/
```

### What You Have:
- `summary.csv` - All results in one table
- `analysis/*.png` - 4 analysis plots
- `run_XXX_*/` - Individual run folders with:
  - `config.json` - Exact hyperparameters
  - `metrics.csv` - Episode-by-episode data
  - `final_model.pkl` - Trained agent

### Next Steps:
1. Download `summary.csv` and analyze in Excel/Python
2. Share plots with your partner
3. Write paper Results section!

---

**Total runtime:** Check the cell execution times above